In [10]:
"""
Some results (e.g. miconic evaluation, or blocksworld training) are either timing out (due to apptainer and cluster cpus) or running out of memory (possibly due to suboptimal handcrafted policies, or very large training datasets).
"""

from _results_helper import *
import warnings
warnings.filterwarnings("ignore")

In [11]:
choices = [
    # "sample",
    "best",
]
layers = [
    1,
    2,
    3,
    4,
    5,
]
dimensions = [
    1,
    # 2,
    # 4,
    # 8,
    16,
    32,
    64,
    # 128,
]

# Baseline Cycle Count

In [12]:
visualise_cylces()

Ideally, we want 0 cycles


# Train Stats

In [13]:
visualise_train()

blocksworld


ferry


satellite


# Quantified improvements

In [14]:
quantify_performance(choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


# Plan length plots

In [6]:
log_y = 0

plot_domains("plan_length", log_y=log_y, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


In [7]:
plot_domains("cycles", log_y=log_y, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


# Absolute difference plots

In [8]:
plot_difference(absolute=True, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


# Relative difference plots

In [9]:
plot_difference(absolute=False, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite
